In [ ]:
import tensorflow as tf
import pandas as pd
import zipfile, os, shutil
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm as tq
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Cek Versi Tensorflow
print(tf.__version__)

2.5.0


In [ ]:
# Download Dataset Katarak
! KAGGLE_CONFIG_DIR=/content/ kaggle datasets download jr2ngb/cataractdataset
! chmod 600 kaggle.json
! ls ~/.kaggle 2>/dev/null || mkdir ~/.kaggle
! mv kaggle.json ~/.kaggle
! kaggle datasets download jr2ngb/cataractdataset

100% 3.33G/3.34G [00:39<00:00, 109MB/s] 
100% 3.34G/3.34G [00:39<00:00, 90.3MB/s]
cataractdataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
# Proses Ekstrak Dataset
with zipfile.ZipFile('cataractdataset.zip','r') as z:
  z.extractall('./')

os.listdir()

['.config',
 'dataset',
 'README.md',
 'cataractdataset.zip',
 'repository',
 'sample_data']

In [ ]:
# Hapus Direktori yang Tidak Digunakan
dir_path = '/content/repository'

shutil.rmtree(dir_path)

In [ ]:
base_dir = '/content/dataset'

file_name = []
tag = []
full_path = []
for path, subdirs, files in os.walk(base_dir):
    for name in files:
        full_path.append(os.path.join(path, name)) 
        tag.append(path.split('/')[-1])        
        file_name.append(name)

In [ ]:
# Menerapkan Fungsi Callback
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')>0.60 and logs.get('val_accuracy')>0.60):
            print("\nAkurasi telah mencapai lebih dari 60%, proses dihentikan!")
            self.model.stop_training = True

In [ ]:
# Proses Split Dataset
df = pd.DataFrame({"path":full_path,'file_name':file_name,"tag":tag})
df.groupby(['tag']).size()

X = df['path']
y = df['tag']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=300)

df_tr = pd.DataFrame({'path':X_train, 'tag':y_train, 'set':'train'})
df_val = pd.DataFrame({'path':X_val, 'tag':y_val, 'set':'validation'})

print('train size', len(df_tr))
print('val size', len(df_val))

df_all = df_tr.append([df_val]).reset_index(drop=1)
print(df_all.groupby(['set','tag']).size(),'\n')

df_all.sample(3)

train size 540
val size 61
set         tag             
train       1_normal            266
            2_cataract           92
            2_glaucoma           92
            3_retina_disease     90
validation  1_normal             34
            2_cataract            8
            2_glaucoma            9
            3_retina_disease     10
dtype: int64 



,path,tag,set
260,/content/dataset/1_normal/NL_136.png,1_normal,train
329,/content/dataset/1_normal/NL_080.png,1_normal,train
442,/content/dataset/2_glaucoma/Glaucoma_060.png,2_glaucoma,train


In [ ]:
datasource_path = base_dir
dataset_path = "content/dataset/dataset/"

for index, row in tq(df_all.iterrows()):
    
    #detect filepath
    file_path = row['path']            
    if os.path.exists(file_path) == False:
            file_path = os.path.join(datasource_path,row['tag'],row['image'].split('.')[0])  
    
    #make folder destination dirs
    if os.path.exists(os.path.join(dataset_path,row['set'],row['tag'])) == False:
        os.makedirs(os.path.join(dataset_path,row['set'],row['tag']))
    
    #define file dest
    destination_file_name = file_path.split('/')[-1]
    file_dest = os.path.join(dataset_path,row['set'],row['tag'],destination_file_name)
    
    #copy file from source to dest
    if os.path.exists(file_dest) == False:
        shutil.copy2(file_path,file_dest)

In [ ]:
# Image Augmentation
train_dir = "content/dataset/dataset/train"
val_dir = "content/dataset/dataset/validation"

train_datagen = ImageDataGenerator(
                  rescale=1./255,
                  rotation_range=40,
                  vertical_flip=True,
                  shear_range = 0.2,
                  zoom_range=0.2,
                  fill_mode = 'nearest')

val_datagen = ImageDataGenerator(
                  rescale=1./255,
                  rotation_range=40,
                  vertical_flip=True,
                  shear_range = 0.2,
                  zoom_range=0.2,
                  fill_mode = 'nearest')

train_generator = train_datagen.flow_from_directory(
        train_dir,  # direktori data latih
        target_size=(150,150),          
        class_mode='categorical')
 
validation_generator = val_datagen.flow_from_directory(
        val_dir, # direktori data validasi
        target_size=(150, 150), 
        class_mode='categorical')

Found 540 images belonging to 4 classes.
Found 61 images belonging to 4 classes.


In [ ]:
# Simple Model CNN
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax')
])

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 34, 34, 64)        36928     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 17, 17, 64)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 15, 15, 128)      

In [ ]:
callbacks = myCallback()
history = model.fit(
    train_generator,
    steps_per_epoch = 15,
    epochs = 100,
    validation_data = validation_generator,
    validation_steps = 2,
    verbose =1,
    callbacks=[callbacks]
)

Epoch 1/100
15/15 [==============================] - 75s 5s/step - loss: 1.3397 - accuracy: 0.4559 - val_loss: 1.2624 - val_accuracy: 0.5574
Epoch 2/100
15/15 [==============================] - 70s 5s/step - loss: 1.3077 - accuracy: 0.4937 - val_loss: 1.2731 - val_accuracy: 0.5574
Epoch 3/100
15/15 [==============================] - 70s 5s/step - loss: 1.2709 - accuracy: 0.4958 - val_loss: 1.2505 - val_accuracy: 0.5574
Epoch 4/100
15/15 [==============================] - 70s 5s/step - loss: 1.2473 - accuracy: 0.4979 - val_loss: 1.2162 - val_accuracy: 0.5574
Epoch 5/100
15/15 [==============================] - 72s 5s/step - loss: 1.2563 - accuracy: 0.4916 - val_loss: 1.2323 - val_accuracy: 0.5574
Epoch 6/100
15/15 [==============================] - 72s 5s/step - loss: 1.2807 - accuracy: 0.4937 - val_loss: 1.1693 - val_accuracy: 0.5574
Epoch 7/100
15/15 [==============================] - 70s 5s/step - loss: 1.2326 - accuracy: 0.4896 - val_loss: 1.1446 - val_accuracy: 0.5574
Epoch 8/100
1

In [ ]:
# Grafik untuk Loss Train dan Loss Validation..
from matplotlib import pyplot as plt

plt.plot(history.history['loss'], label ='Training loss')
plt.plot(history.history['val_loss'], label = 'Validation loss')
plt.title('Loss Plot')
plt.ylabel('Value')
plt.xlabel('Epoch')
plt.legend(loc='upper right')
plt.show()

In [ ]:
# Grafik untuk Akurasi Train dan Akurasi validation..
plt.plot(history.history['accuracy'], label = 'Train Accuracy')
plt.plot(history.history['val_accuracy'], label = 'Validation Accuracy')
plt.title('Accuracy Plot')
plt.ylabel('Value')
plt.xlabel('Epoch')
plt.legend(loc='lower right')
plt.show()

In [ ]:
tf.keras.models.save_model(model,'my_model.hdf5')

In [ ]:
# Konversi model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with tf.io.gfile.GFile('model.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpl9497g1v/assets
